# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 2:57PM,248583,10,0086074697,ISDIN Corporation,Released
1,Oct 13 2022 2:57PM,248583,10,0086074698,ISDIN Corporation,Released
2,Oct 13 2022 2:57PM,248583,10,0086074705,ISDIN Corporation,Released
3,Oct 13 2022 2:57PM,248583,10,0086074707,ISDIN Corporation,Released
4,Oct 13 2022 2:57PM,248583,10,0086074711,ISDIN Corporation,Released
5,Oct 13 2022 2:57PM,248583,10,0086074712,ISDIN Corporation,Released
6,Oct 13 2022 2:57PM,248583,10,0086074717,ISDIN Corporation,Released
7,Oct 13 2022 2:57PM,248583,10,0086074716,ISDIN Corporation,Released
8,Oct 13 2022 2:57PM,248583,10,0086074718,ISDIN Corporation,Released
9,Oct 13 2022 2:57PM,248583,10,0086074724,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,248576,Released,1
21,248579,Executing,1
22,248579,Released,5
23,248582,Released,1
24,248583,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248572,NaN,NaN,1.0
248576,NaN,NaN,1.0
248579,NaN,1.0,5.0
248582,NaN,NaN,1.0
248583,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248089,0.0,0.0,1.0
248096,0.0,0.0,1.0
248101,0.0,0.0,1.0
248103,0.0,0.0,1.0
248109,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248089,0,0,1
248096,0,0,1
248101,0,0,1
248103,0,0,1
248109,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248089,0,0,1
1,248096,0,0,1
2,248101,0,0,1
3,248103,0,0,1
4,248109,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248089,,,1
1,248096,,,1
2,248101,,,1
3,248103,,,1
4,248109,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 2:57PM,248583,10,ISDIN Corporation
12,Oct 13 2022 2:52PM,248582,10,Yusen – 3D Matrix
13,Oct 13 2022 2:33PM,248579,10,Emerginnova
19,Oct 13 2022 2:13PM,248109,21,"NBTY Global, Inc."
20,Oct 13 2022 2:12PM,248103,21,"NBTY Global, Inc."
21,Oct 13 2022 2:12PM,248101,21,"NBTY Global, Inc."
22,Oct 13 2022 2:12PM,248096,21,"NBTY Global, Inc."
23,Oct 13 2022 2:11PM,248554,21,"NBTY Global, Inc."
24,Oct 13 2022 2:10PM,248576,19,"Methapharm, Inc."
25,Oct 13 2022 2:10PM,248552,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 13 2022 2:57PM,248583,10,ISDIN Corporation,,,12
1,Oct 13 2022 2:52PM,248582,10,Yusen – 3D Matrix,,,1
2,Oct 13 2022 2:33PM,248579,10,Emerginnova,,1,5
3,Oct 13 2022 2:13PM,248109,21,"NBTY Global, Inc.",,,1
4,Oct 13 2022 2:12PM,248103,21,"NBTY Global, Inc.",,,1
5,Oct 13 2022 2:12PM,248101,21,"NBTY Global, Inc.",,,1
6,Oct 13 2022 2:12PM,248096,21,"NBTY Global, Inc.",,,1
7,Oct 13 2022 2:11PM,248554,21,"NBTY Global, Inc.",,,1
8,Oct 13 2022 2:10PM,248576,19,"Methapharm, Inc.",,,1
9,Oct 13 2022 2:10PM,248552,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 2:57PM,248583,10,ISDIN Corporation,12,,
1,Oct 13 2022 2:52PM,248582,10,Yusen – 3D Matrix,1,,
2,Oct 13 2022 2:33PM,248579,10,Emerginnova,5,1,
3,Oct 13 2022 2:13PM,248109,21,"NBTY Global, Inc.",1,,
4,Oct 13 2022 2:12PM,248103,21,"NBTY Global, Inc.",1,,
5,Oct 13 2022 2:12PM,248101,21,"NBTY Global, Inc.",1,,
6,Oct 13 2022 2:12PM,248096,21,"NBTY Global, Inc.",1,,
7,Oct 13 2022 2:11PM,248554,21,"NBTY Global, Inc.",1,,
8,Oct 13 2022 2:10PM,248576,19,"Methapharm, Inc.",1,,
9,Oct 13 2022 2:10PM,248552,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 2:57PM,248583,10,ISDIN Corporation,12,,
1,Oct 13 2022 2:52PM,248582,10,Yusen – 3D Matrix,1,,
2,Oct 13 2022 2:33PM,248579,10,Emerginnova,5,1,
3,Oct 13 2022 2:13PM,248109,21,"NBTY Global, Inc.",1,,
4,Oct 13 2022 2:12PM,248103,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 2:57PM,248583,10,ISDIN Corporation,12.0,NaN,NaN
1,Oct 13 2022 2:52PM,248582,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Oct 13 2022 2:33PM,248579,10,Emerginnova,5.0,1.0,NaN
3,Oct 13 2022 2:13PM,248109,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Oct 13 2022 2:12PM,248103,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 2:57PM,248583,10,ISDIN Corporation,12.0,0.0,0.0
1,Oct 13 2022 2:52PM,248582,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Oct 13 2022 2:33PM,248579,10,Emerginnova,5.0,1.0,0.0
3,Oct 13 2022 2:13PM,248109,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Oct 13 2022 2:12PM,248103,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1988518,41.0,18.0,9.0
15,496806,30.0,8.0,37.0
19,497108,2.0,0.0,0.0
21,1737604,7.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1988518,41.0,18.0,9.0
1,15,496806,30.0,8.0,37.0
2,19,497108,2.0,0.0,0.0
3,21,1737604,7.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,41.0,18.0,9.0
1,15,30.0,8.0,37.0
2,19,2.0,0.0,0.0
3,21,7.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,41.0
1,15,Released,30.0
2,19,Released,2.0
3,21,Released,7.0
4,10,Executing,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21
Status,,,,
Completed,9.0,37.0,0.0,0.0
Executing,18.0,8.0,0.0,0.0
Released,41.0,30.0,2.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21
0,Completed,9.0,37.0,0.0,0.0
1,Executing,18.0,8.0,0.0,0.0
2,Released,41.0,30.0,2.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21
0,Completed,9.0,37.0,0.0,0.0
1,Executing,18.0,8.0,0.0,0.0
2,Released,41.0,30.0,2.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()